In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Step 1

In [2]:
data = tf.keras.utils.get_file(cache_dir=".", origin="https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip", extract=True)

13565642/13565642 [==============================] - 1s 0us/step


In [3]:
!pwd

/content


In [4]:
df = pd.read_csv("./datasets/jena_climate_2009_2016.csv")
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [5]:
temperature = np.array(df['T (degC)'].values)
raw_data = np.array(df.drop(['Date Time'], axis=1).values)

# Step 2

In [6]:
perc_50 = len(raw_data) // 2
perc_25 = len(raw_data) // 4
perc_50, perc_25

(210225, 105112)

In [7]:
mean = raw_data[:perc_50].mean(axis=0)
sigma = raw_data[:perc_50].std(axis=0)

mean, sigma

(array([ 988.74929466,    8.82590329,  282.9050718 ,    4.31331863,
          75.87275476,   13.14569946,    9.19414209,    3.95148184,
           5.81050741,    9.30208943, 1218.45204015,    2.14977462,
           3.56048029,  176.4405232 ]),
 array([ 8.50510865,  8.77098361,  8.86560463,  7.08011256, 16.62877917,
         7.6012955 ,  4.14689697,  4.76961017,  2.63277288,  4.1996886 ,
        42.0392747 ,  1.53358803,  2.32305586, 85.85032063]))

In [8]:
raw_data[0]

array([ 9.96520e+02, -8.02000e+00,  2.65400e+02, -8.90000e+00,
        9.33000e+01,  3.33000e+00,  3.11000e+00,  2.20000e-01,
        1.94000e+00,  3.12000e+00,  1.30775e+03,  1.03000e+00,
        1.75000e+00,  1.52300e+02])

In [9]:
raw_data = (raw_data - mean) / sigma
raw_data[0]

array([ 0.91365151, -1.92064015, -1.97449272, -1.86625827,  1.04801712,
       -1.29131928, -1.46715535, -0.78234525, -1.47012583, -1.4720352 ,
        2.12415558, -0.73016651, -0.77935289, -0.28119316])

In [10]:
def create_ds(raw_data, target, shuffle=True,batch_size=256):
    sampling_rate=6
    sequence_length=120
    delay=sampling_rate * (sequence_length + 24 -1)
    return tf.keras.utils.timeseries_dataset_from_array(
        data=raw_data,
        targets=target[delay:],
        sampling_rate=sampling_rate,
        sequence_length=sequence_length,
        shuffle=shuffle,
        batch_size=batch_size
    )

In [ ]:
my_check_ds = create_ds(
  raw_data=df. values [: 1000, 0],
  target=df. values [: 1000, 0],
  shuffle=False,
  batch_size=1
)

for features, target in my_check_ds.take(2):
  print (f"{features} maps to {target}")
  print("*"*50)

In [11]:
train_ds = create_ds(raw_data[:perc_50], temperature[:perc_50])
val_ds = create_ds(raw_data[perc_50:-perc_25], temperature[perc_50: -perc_25])
test_ds = create_ds(raw_data[-perc_25:], temperature[-perc_25:])

In [12]:
for samples, targets in train_ds.take(1):
    print(f"samples shape: {samples.shape}")
    print(f"targets shape: {targets.shape}")

samples shape: (256, 120, 14)
targets shape: (256,)


# Step 3

In [13]:
def compute_mae_naive(dataset):
  mu = temperature[:perc_50].mean()
  std = temperature[:perc_50].std()

  total_error = 0.0
  total_samples = 0

  for item, label in dataset:
    unnormal = item[:, -1, 1] * std + mu
    total_error += np.sum(np.abs(label - unnormal))
    total_samples += item.shape[0]

  return total_error / total_samples

In [ ]:
compute_mae_naive(val_ds), compute_mae_naive(test_ds)

(2.440577520718234, 2.621074299307458)

# Step 4

In [14]:
for item, label in train_ds.take(1):
  print(item.shape)

(256, 120, 14)


In [19]:
model_cp_cb = tf.keras.callbacks.ModelCheckpoint("jena_climate.keras", save_best_only=True)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[120, 14]),
    tf.keras.layers.Dense(units=16, activation="relu"),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 1680)              0         
                                                                 
 dense_2 (Dense)             (None, 16)                26896     
                                                                 
 dense_3 (Dense)             (None, 10)                170       
                                                                 
Total params: 27066 (105.73 KB)
Trainable params: 27066 (105.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

In [21]:
train_ds = train_ds.cache().prefetch(1)
val_ds = val_ds.cache().prefetch(1)

model.fit(train_ds, epochs=10, callbacks=[model_cp_cb], validation_data=val_ds)

Epoch 1/10
818/818 [==============================] - 5s 5ms/step - loss: 20.8254 - mae: 3.2677 - val_loss: 10.8057 - val_mae: 2.5877
Epoch 2/10
818/818 [==============================] - 4s 5ms/step - loss: 9.5940 - mae: 2.4488 - val_loss: 10.6938 - val_mae: 2.5728
Epoch 3/10
818/818 [==============================] - 4s 5ms/step - loss: 9.1765 - mae: 2.3948 - val_loss: 10.8190 - val_mae: 2.5877
Epoch 4/10
818/818 [==============================] - 5s 6ms/step - loss: 8.9345 - mae: 2.3634 - val_loss: 10.9184 - val_mae: 2.5982
Epoch 5/10
818/818 [==============================] - 4s 5ms/step - loss: 8.7541 - mae: 2.3393 - val_loss: 11.2206 - val_mae: 2.6358
Epoch 6/10
818/818 [==============================] - 4s 5ms/step - loss: 8.6200 - mae: 2.3215 - val_loss: 11.2971 - val_mae: 2.6473
Epoch 7/10
818/818 [==============================] - 4s 5ms/step - loss: 8.5224 - mae: 2.3084 - val_loss: 11.1257 - val_mae: 2.6288
Epoch 8/10
818/818 [==============================] - 4s 5ms/step - 

In [ ]:
!ls -la

total 344
drwxr-xr-x 1 root root   4096 Jan 13 08:06 .
drwxr-xr-x 1 root root   4096 Jan 13 07:55 ..
drwxr-xr-x 2 root root   4096 Jan 13 08:02 assets
drwxr-xr-x 4 root root   4096 Jan 11 17:01 .config
drwxr-xr-x 3 root root   4096 Jan 13 07:56 datasets
-rw-r--r-- 1 root root     56 Jan 13 08:03 fingerprint.pb
-rw-r--r-- 1 root root 237331 Jan 13 08:07 jena_climate.keras
-rw-r--r-- 1 root root   7800 Jan 13 08:03 keras_metadata.pb
drwxr-xr-x 1 root root   4096 Jan 11 17:02 sample_data
-rw-r--r-- 1 root root  70949 Jan 13 08:03 saved_model.pb
drwxr-xr-x 2 root root   4096 Jan 13 08:03 variables


In [ ]:
!pwd

/content


In [23]:
model = tf.keras.models.load_model("jena_climate.keras")
model.evaluate(test_ds)

408/408 [==============================] - 14s 33ms/step - loss: 11.3399 - mae: 2.6644


[11.339898109436035, 2.6643593311309814]

# Step 5

In [25]:
model_cp_cb = tf.keras.callbacks.ModelCheckpoint("jena_climate1.keras", save_best_only=True)

model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(16, input_shape=[None, 14]),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

model.fit(train_ds, epochs=10, callbacks=[model_cp_cb], validation_data=val_ds)

Epoch 1/10
818/818 [==============================] - 68s 81ms/step - loss: 46.9184 - mae: 5.0292 - val_loss: 13.2035 - val_mae: 2.7058
Epoch 2/10
818/818 [==============================] - 66s 81ms/step - loss: 11.4466 - mae: 2.6242 - val_loss: 10.0510 - val_mae: 2.4283
Epoch 3/10
818/818 [==============================] - 67s 82ms/step - loss: 10.5200 - mae: 2.5291 - val_loss: 9.1823 - val_mae: 2.3541
Epoch 4/10
818/818 [==============================] - 66s 81ms/step - loss: 10.3655 - mae: 2.5088 - val_loss: 9.0969 - val_mae: 2.3457
Epoch 5/10
818/818 [==============================] - 65s 80ms/step - loss: 10.2173 - mae: 2.4893 - val_loss: 9.0215 - val_mae: 2.3334
Epoch 6/10
818/818 [==============================] - 66s 81ms/step - loss: 10.1238 - mae: 2.4770 - val_loss: 9.0392 - val_mae: 2.3364
Epoch 7/10
818/818 [==============================] - 66s 80ms/step - loss: 10.0422 - mae: 2.4669 - val_loss: 9.0217 - val_mae: 2.3345
Epoch 8/10
818/818 [==============================] -

In [26]:
model = tf.keras.models.load_model("jena_climate1.keras")
model.evaluate(test_ds)

408/408 [==============================] - 17s 41ms/step - loss: 10.2091 - mae: 2.4827


[10.209083557128906, 2.482748031616211]

# Step 6

In [ ]:
model_cp_cb = tf.keras.callbacks.ModelCheckpoint("jena_climate2.keras", save_best_only=True)

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=16, input_shape=[None, 14]),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

model.fit(train_ds, epochs=10, callbacks=[model_cp_cb], validation_data=val_ds)

Epoch 1/10
818/818 [==============================] - 11s 11ms/step - loss: 40.6954 - mae: 4.6178 - val_loss: 12.2159 - val_mae: 2.6543
Epoch 2/10
818/818 [==============================] - 8s 9ms/step - loss: 10.6474 - mae: 2.5317 - val_loss: 9.8223 - val_mae: 2.4381
Epoch 3/10
818/818 [==============================] - 8s 9ms/step - loss: 9.6666 - mae: 2.4150 - val_loss: 9.9232 - val_mae: 2.4428
Epoch 4/10
818/818 [==============================] - 8s 10ms/step - loss: 9.3166 - mae: 2.3651 - val_loss: 9.8448 - val_mae: 2.4351
Epoch 5/10
818/818 [==============================] - 8s 10ms/step - loss: 9.0931 - mae: 2.3354 - val_loss: 9.8340 - val_mae: 2.4347
Epoch 6/10
818/818 [==============================] - 8s 9ms/step - loss: 8.8962 - mae: 2.3071 - val_loss: 10.1621 - val_mae: 2.4600
Epoch 7/10
818/818 [==============================] - 8s 10ms/step - loss: 8.7792 - mae: 2.2881 - val_loss: 9.9013 - val_mae: 2.4278
Epoch 8/10
818/818 [==============================] - 8s 10ms/step 

In [ ]:
model = tf.keras.models.load_model("jena_climate2.keras")
model.evaluate(test_ds)

408/408 [==============================] - 13s 31ms/step - loss: 10.7393 - mae: 2.5586


[10.73933219909668, 2.5585954189300537]

# Step 7

In [ ]:
model_cp_cb = tf.keras.callbacks.ModelCheckpoint("jena_climate3.keras", save_best_only=True)

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=32, recurrent_dropout=0.25, input_shape=[None, 14]),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

model.fit(train_ds, epochs=10, callbacks=[model_cp_cb], validation_data=val_ds)

Epoch 1/10
818/818 [==============================] - 382s 320ms/step - loss: 25.7618 - mae: 3.7669 - val_loss: 9.7155 - val_mae: 2.4255
Epoch 2/10
818/818 [==============================] - 261s 319ms/step - loss: 14.8112 - mae: 2.9882 - val_loss: 9.3298 - val_mae: 2.3751
Epoch 3/10
818/818 [==============================] - 265s 324ms/step - loss: 13.9183 - mae: 2.8975 - val_loss: 9.2569 - val_mae: 2.3549
Epoch 4/10
818/818 [==============================] - 262s 320ms/step - loss: 13.2535 - mae: 2.8304 - val_loss: 9.2151 - val_mae: 2.3420
Epoch 5/10
818/818 [==============================] - 268s 328ms/step - loss: 12.7159 - mae: 2.7723 - val_loss: 9.2707 - val_mae: 2.3515
Epoch 6/10
818/818 [==============================] - 257s 315ms/step - loss: 12.3824 - mae: 2.7343 - val_loss: 9.0680 - val_mae: 2.3367
Epoch 7/10
818/818 [==============================] - 264s 323ms/step - loss: 11.9819 - mae: 2.6883 - val_loss: 9.2136 - val_mae: 2.3518
Epoch 8/10
818/818 [=====================

In [ ]:
model = tf.keras.models.load_model("jena_climate3.keras")
model.evaluate(test_ds)

408/408 [==============================] - 28s 67ms/step - loss: 10.7110 - mae: 2.5640


[10.711042404174805, 2.56398606300354]